# [실습1] 이미지 데이터 확인하기

* Numpy, PIL, tensorflow.keras 등을 이용하여 이미지를 Numpy 배열로 바꿔보고, 이를 통해 이미지가 어떻게 이루어졌는지 확인하기
* import PIL: 이미지를 불러오고 처리하기 위한 라이브러리
* PIL.Image.open(path): 이미지 불러오기
* PIL.Image.resize(width, height): 이미지 크기 조정하기

In [1]:
import  pandas as pd
import numpy as np

import PIL
import matplotlib.image as img
import matplotlib.pyplot as plt

#from elice_utils import EliceUtils
#elice_utils = EliceUtils()

In [2]:
# 이미지 목록을 불러오는 함수입니다.

def load_data(path):
    return pd.read_csv(path)

In [3]:
'''
지시사항 1번
   PIL.Image를 이용하여 
   이름(경로+이름)을 바탕으로 이미지를 불러오고,
   이를 리스트 'images'에 추가하는 함수를 완성합니다.
   main 함수에서 'path'와 'names' 변수를 확인해보세요.
'''

def load_images(path, names):
    
    images=[]
    for fname in names:
        file_path = path + fname # 이미지 경로 지정
        im = PIL.Image.open(file_path) # 이미지 불러오기
        images.append(im) # 이미지 리스트에 추가
    
    return images

In [4]:
'''
지시사항 2번
   이미지의 사이즈를 main 함수에 있는 'IMG_SIZE'로 
   조정하고, 이를 Numpy 배열로 변환하는 함수를 완성합니다.
'''

def images2numpy(images, size):
    
    output = []
    for im in images:
        im_resize = im.resize(size) # 이미지 크기 조정
        im_npy = np.array(im_resize) # 이미지 to 넘파이
        output.append(im_npy) # 아웃풋 리스트에 추가
        
    output = np.array(output) # 최종 아웃풋도 넘파이 형태 
    return output

In [5]:
# 이미지에 대한 정보를 나타내주는 함수입니다.

def sampleVisualize(np_images):

    fileName = "./data/images/1000092795.jpg"
    ndarray = img.imread(fileName)
    
    plt.imshow(ndarray)
    plt.show()    
    plt.savefig("plot.png")
    
    print("\n1-1. 'fileName' 이미지(원본): ")
    elice_utils.send_image("plot.png")
    
    print('\n1-2. Numpy array로 변환된 원본 이미지:', ndarray)
    print('\n1-3. Numpy array로 변환된 원본 이미지의 크기:', np.array(ndarray).shape)
    
    plt.imshow(np_images[0])
    plt.show()
    plt.savefig("plot_re.png")
    
    print("\n2-1. 'fileName' 이미지(resize 후): ")
    elice_utils.send_image("plot_re.png")
    
    print('\n2-2. Numpy array로 변환된 resize 후 이미지:', np_images[0])
    print('\n2-3. Numpy array로 변환된 resize 후 이미지 크기:', np.array(np_images[0]).shape)    
    
    print('\n3. Numpy array로 변환된 resize 후 이미지 10장의 크기:', np.array(np_images).shape)

In [14]:
'''
지시사항 3번
   main 함수를 완성하세요.
   
   Step01. 이미지를 불러오는 함수를 이용해 
           'images'를 정의합니다.
   
   Step02. 이미지를 Numpy 배열로 바꾸는 함수를 이용해
           'np_images'를 정의합니다.
'''

def main():
    
    CSV_PATH = "./data/data.csv"
    IMG_PATH = "./data/images/"
    IMG_SIZE = (300,300)
    MAX_LEN = 30
    BATCH_SIZE = 2
    
    name_caption = load_data(CSV_PATH)
    names = name_caption['file_name']
    print(names)
    
    images = load_images(path=IMG_PATH, names=names) # 이미지 불러오기
    np_images = images2numpy(images, size=IMG_SIZE) # 이미지 넘파이로 변환하기
    
    sampleVisualize(np_images)
    
    return images, np_images
    
if __name__=='__main__':
    main()

# [실습2] 일반 다층 퍼셉트론 모델(MLP)로 이미지 데이터 분류하기

* 테스트용 MNIST 데이터를 96% 이상의 정확도로 분류하는 MLP모델을 만들고 학습하기
* tf.keras.layers.Flatten(): 2차원 데이터를 1차원 데이터로 평평하게 만들어줌
* tf.keras.layers.Dense(node, activation): MLP의 기본 layer
* from tensorflow.keras.utils import to_categorical: label을 클래스화 하는 원-핫 인코딩 모듈
* label = to_categorical(label, 10): 원-핫 인코딩

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from visual import *

import logging, os
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

# 동일한 실행 결과 확인을 위한 코드입니다.
np.random.seed(123)
tf.random.set_seed(123)

In [ ]:
'''
지시사항 1번
   MNIST 데이테 셋을 전처리하는 'preprocess' 함수를 완성합니다.

   Step01. MNIST 데이터 이미지를 0~1 사이 값으로 정규화해줍니다.
           원본은 0~255 사이의 값입니다.
           
   Step02. 0~9 사이 값인 label을 클래스화 하기 위해 
           원-핫 인코딩을 진행합니다.

'''

def preprocess():
    
    # MNIST 데이터 세트를 불러옵니다.
    mnist = tf.keras.datasets.mnist
    
    # MNIST 데이터 세트를 Train set과 Test set으로 나누어 줍니다.
    (train_images, train_labels), (test_images, test_labels) = mnist.load_data()    
    
    # 이미지 정규화
    train_images = train_images/255.
    test_images = test_images/255.
    
    # 원-핫 인코딩
    train_labels = to_categorical(train_labels, 10)
    test_labels = to_categorical(test_labels, 10)
    
    return train_images, test_images, train_labels, test_labels

In [ ]:
'''
지시사항 2번
   다층 퍼셉트론(MLP) 모델을 생성합니다.

'''
def MLP():
    
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Flatten(input_shape=(28,28))) # input layer
    model.add(tf.keras.layers.Dense(128, activation='relu'))
    model.add(tf.keras.layers.Dense(256, activation='relu'))
    model.add(tf.keras.layers.Dense(10, activation='softmax'))  # output layer
    
    return model

In [ ]:
'''
지시사항 3번
   모델을 불러온 후 학습시키고 테스트 데이터에 대해 평가합니다.

   Step01. MLP 함수를 통해 모델을 불러옵니다.
   
   Step02. 모델의 손실 함수, 최적화 알고리즘, 평가 방법을 설정합니다.
   
   Step03. 모델의 구조를 확인하는 코드를 작성합니다.
   
   Step04. 모델을 학습시킵니다. 검증용 데이터도 설정하세요.
           'epochs'와 'batch_size'도 자유롭게 설정하세요.
              
   Step05. 모델을 테스트하고 손실(loss)값과 
           Test Accuracy 값 및 예측 클래스, 
           손실 함수값 그래프를 출력합니다. 
           
           모델의 성능을 확인해보고,
           목표값을 달성해보세요.
'''

def main():
    
    # 데이터를 불러옵니다.
    train_images, test_images, train_labels, test_labels = preprocess()
    
    # 지시사항 2에서 설정한 모델을 불러옵니다.
    model = MLP()
    
    # 모델의 구조를 확인합니다.
    model.summary()
    
    # 컴파일러를 설정합니다.
    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=['accuracy'])
    
    # fit 함수를 사용하여 모델을 학습합니다.
    # 학습 수행 시 정보는 history에 저장합니다.
    history = model.fit(train_images, train_labels, epochs=6, batch_size=128, verbose=2,
                        validation_data = (test_images, test_labels))
    
    # evaluate 함수를 사용하여 테스트 데이터의 결과값을 저장합니다.
    loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
    
    print('\nTest Loss : {:.4f} | Test Accuracy : {}'.format(loss, test_acc))
    print('예측한 Test Data 클래스 : ',model.predict_classes(test_images))
    
    Visulaize([('MLP', history)], 'loss')
    
    return test_acc


if __name__ == "__main__":
    main()

# [실습3] Convolution Layer

* Convolution Layer를 구성하는 방법 알아보기
* keras.layers.Conv2D(filters, kernal_size, strides, padding, activation)(image)

 -filters: output filter의 개수
 
 -kernel_size: Convolution을 위한 kernel    
 
 -strides: 정수 및 튜플, 리스트 형태의 값 ex) (2,2)      
 
 -padding: SAME(패딩 O) / VALID(패딩 X)
 
 -activation: 활성화 함수 ex) relu, softmax
 
 -image: 입력 영상

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from elice_utils import EliceUtils
elice_utils = EliceUtils()

In [ ]:
def Visualize(image, x, y):
    plt.imshow(image.reshape(x,y), cmap ='Greys')
    plt.savefig('plot.png')
    elice_utils.send_image("plot.png")

In [ ]:
# 임의의 3 x 3 x 1 영상을 하나 만들어줍니다.
image = np.array([[[[1],[2],[3]],
                   [[4],[5],[6]],
                   [[7],[8],[9]]]], dtype = np.float32)

# 합성곱 연산을 위해 임의의 2 x 2 x 1 커널을 하나 만들어줍니다.
kernel = np.array([[[[1.]],[[1.]]],
                      [[[1.]],[[1.]]]])

In [ ]:
# 이미지 Shape 출력 : (num of image, width, height, channel)
print('Image shape : ', image.shape)
# 커널 Shape 출력 : (width, height, channel, num of kernel)
print('Kernel shape : ', kernel.shape)
# tf.nn.conv2d에 넣기 위해 이미지와 커널의 Shape을 위와 같이 만들었습니다.

In [ ]:
# Gray 이미지 출력
Visualize(image, 3 ,3)

In [ ]:
kernel_init = tf.constant_initializer(kernel)
# Convolution Layer 선언
'''
지시사항1번 
   keras.layers.Conv2D()를 완성하세요.
'''
conv2d = keras.layers.Conv2D(filters=1, kernel_size=2, padding='VALID', kernel_initializer=kernel_init)(image)
Visualize(conv2d.numpy(), 2,2)

# [실습4] Max Pooling Layer

* Pooling Layer: Convolution Layer의 결과(Feature Map)를 축소시키는데 사용하는 층
* Feature map 사이즈 다운 -> 연산량 감소
* Feature 개수 감소 -> Overfitting 방지
* keras.layers.MaxPool2D(pool_size, strides, padding)

 -pool_size: Pooling filter의 크기
 
 -strides: Filter를 적용할 간격
 
 -padding: SAME or VALID

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from PIL import Image
from tensorflow import keras
from elice_utils import EliceUtils
elice_utils = EliceUtils()

In [ ]:
def Visualize(image, x, y):
    plt.imshow(image.reshape(x,y), cmap ='Greys')
    plt.savefig('plot.png')
    elice_utils.send_image("plot.png")

In [ ]:
# 임의의 3 x 3 x 1 영상을 하나 만들어줍니다.
image = tf.constant([[[[1],[2],[3],[4]],
                   [[4],[5],[6],[7]],
                   [[7],[8],[9],[10]],
                   [[3],[5],[7],[9]]]], dtype = np.float32)

In [ ]:
'''
지시사항 1번
Max Pooling Layer를 선언하세요.
''' 
pool = tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding='VALID')(image)

print(pool.shape)
print(pool.numpy())

In [ ]:
# 원본 영상과 Max Pooling 후 영상을 출력합니다..
Visualize(image.numpy(), 4,4)
Visualize(pool.numpy(), 2,2)

# [실습5] Keras로 CNN 구현하기

* 테스트용 MNIST 데이터를 95% 이상의 정확도로 분류하는 CNN 모델 만들기
* tf.keras.layers.Conv2D(filters, kernel_size, activation, padding): 입력 이미지의 피처맵을 추출하는 레이어
* tf.keras.layers.MaxPool2D(padding): 처리할 피처맵의 크기를 줄여주는 레이어
* tf.keras.layers.Flatten(): Conv/Pooling의 결과는 N-차원의 텐서 형태, 이를 1차원으로 평평하게 만들어줌
* tf.keras.layers.Dense(node, activation): Fully Connected Layer(=MLP)
* np.expand_dims(data, axis): Numpy 배열 데이터에서 마지막 축(axis)에 해당하는 곳에 차원 하나 추가

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from visual import *
from plotter import *

import logging, os
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

# 동일한 실행 결과 확인을 위한 코드입니다.
np.random.seed(123)
tf.random.set_seed(123)

In [ ]:
'''
지시사항 1번
MNIST 데이테 셋을 전처리하는 'preprocess' 함수를 완성합니다.
   
   Step01과 Step03은 이전 실습과 동일한 코드를 사용할 수 있습니다.

   Step01. MNIST 데이터 이미지를 0~1 사이 값으로 정규화해줍니다.
           원본은 0~255 사이의 값입니다.
           
   Step02. MNIST 데이터의 채널 차원을 추가해줍니다.
           
   Step03. 0~9 사이 값인 레이블을 클래스화 하기 위해 원-핫 인코딩을 진행합니다.
   
'''

def preprocess():
    
    # MNIST 데이터 세트를 불러옵니다.
    mnist = tf.keras.datasets.mnist
    
    # MNIST 데이터 세트를 Train set과 Test set으로 나누어 줍니다.
    (train_images, train_labels), (test_images, test_labels) = mnist.load_data()    
    
    # Train 데이터 5000개와 Test 데이터 1000개를 사용합니다.
    train_images, train_labels = train_images[:5000], train_labels[:5000]
    test_images, test_labels = test_images[:1000], test_labels[:1000]
    
    # Step01. 이미지 정규화
    train_images = train_images/255.
    test_images = test_images/255.
    
    # Step02. 데이터 차원 추가 for Conv
    train_images = np.expand_dims(train_images, axis=-1)
    test_images = np.expand_dims(test_images, axis=-1)
    
    # Step03. 원-핫 인코딩
    train_labels = to_categorical(train_labels, 10)
    test_labels = to_categorical(test_labels, 10)
    
    return train_images, test_images, train_labels, test_labels

In [ ]:
'''
지시사항 2번
CNN 모델을 생성합니다.
'''
def CNN():
    
    # 모델 구축
    model = tf.keras.Sequential()
    
    # input layer
    model.add(tf.keras.layers.Input(shape=(28, 28, 1))) 
    
    # conv + pooling
    model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', padding='SAME'))
    model.add(tf.keras.layers.MaxPool2D(2, padding='VALID'))
    model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu', padding='SAME'))
    model.add(tf.keras.layers.MaxPool2D(2, padding='VALID'))
    model.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation='relu', padding='SAME'))
    model.add(tf.keras.layers.MaxPool2D(2, padding='VALID'))
    
    # output layer(MLP)
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(16, activation='relu'))
    model.add(tf.keras.layers.Dense(10, activation='softmax'))
    
    return model

In [ ]:
'''
지시사항 3번
모델을 불러온 후 학습시키고 테스트 데이터에 대해 평가합니다.

   Step01. CNN 함수를 통해 모델을 불러옵니다.
   
   Step02. 모델의 손실 함수, 최적화 알고리즘, 평가 방법을 설정합니다.
   
   Step03. 모델의 구조를 확인하는 코드를 작성합니다.
   
   Step04. 모델을 학습시킵니다. 검증용 데이터도 설정하세요.
           'epochs'와 'batch_size'도 자유롭게 설정하세요.
           단, 'epochs'이 클수록, 'batch_size'는 작을수록 학습 속도가 느립니다.
   
   Step05. 모델을 테스트하고 손실(loss)값과 Test Accuracy 값 및 예측 클래스, 
           손실 함수값 그래프를 출력합니다. 모델의 성능을 확인해보고,
           목표값을 달성해보세요.
'''

def main():
    
    # 데이터를 불러옵니다.
    train_images, test_images, train_labels, test_labels = preprocess()
    
    # 지시사항 2에서 설정한 모델을 불러옵니다.
    model = CNN()
    
    # 모델의 구조를 확인합니다.
    model.summary()
    
    # 컴파일러를 설정합니다.
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    # fit 함수를 사용하여 모델을 학습합니다.
    # 학습 수행 시 정보는 history에 저장합니다.
    history = model.fit(train_images, train_labels, epochs=6, batch_size=128, verbose=2,
                        validation_data=(test_images, test_labels))
    
    # evaluate 함수를 사용하여 테스트 데이터의 결과값을 저장합니다.
    loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
    
    print('\nTest Loss : {:.4f} | Test Accuracy : {}'.format(loss, test_acc))
    print('예측한 Test Data 클래스 : ',model.predict_classes(test_images)[:10])
    
    Visulaize([('CNN', history)], 'loss')
    
    Plotter(test_images, model)
    
    return test_acc
    
if __name__ == "__main__":
    main()